In [ ]:
# pip install ipython-autotime

In [34]:
import time
import chess
from IPython.display import display, HTML, clear_output
import numpy as np
import pandas as pd
import timeit

# %load_ext autotime

time: 699 µs


### select a random player 

In [2]:
import random
def random_player(board):
    move = random.choice(list(board.legal_moves))
    return move.uci()

time: 863 µs


### helper functions

In [3]:
def who(player):
    return "White" if player == chess.WHITE else "Black"

time: 920 µs


In [4]:
def display_board(board, use_svg):
    if use_svg:
        return board._repr_svg_()
    else:
        return "<pre>" + str(board) + "</pre>"

time: 920 µs


In [61]:
def play_game(player1, player2, visual="svg", pause=0.1):
    """
    playerN1, player2: functions that takes board, return uci move
    visual: "simple" | "svg" | None
    """
    use_svg = (visual == "svg")
    board = chess.Board()
    try:
        while not board.is_game_over(claim_draw=True):
            if board.turn == chess.WHITE:
                uci = player1(board)
            else:
                uci = player2(board)
            name = who(board.turn)
            board.push_uci(uci)
            board_stop = display_board(board, use_svg)
            html = "<b>Move %s %s, Play '%s':</b><br/>%s" % (
                       len(board.move_stack), name, uci, board_stop)
            if visual is not None:
                if visual == "svg":
                    clear_output(wait=True)
                display(HTML(html))
                if visual == "svg":
                    time.sleep(pause)
    except KeyboardInterrupt:
        msg = "Game interrupted!"
        return (False, msg, board)
    result = False
    if board.is_checkmate():
        msg = "checkmate: " + who(not board.turn) + " wins!"
        result = not board.turn
    elif board.is_stalemate():
        msg = "draw: stalemate"
    elif board.is_fivefold_repetition():
        msg = "draw: 5-fold repetition"
    elif board.is_insufficient_material():
        msg = "draw: insufficient material"
    elif board.can_claim_draw():
        msg = "draw: claim"
    if visual is not None:
        print(msg)
    
    return (game_has_winner, msg, board)

time: 3.17 ms


In [62]:
def get_move(prompt):
    uci = input(prompt)
    if uci and uci[0] == "q":
        raise KeyboardInterrupt()
    try:
        chess.Move.from_uci(uci)
    except:
        uci = None
    return uci

time: 834 µs


In [63]:
def count_pieces(board):
    num_pieces = [0,0]
    
    num_pieces[0] += len(board.pieces(chess.PAWN,   chess.WHITE))
    num_pieces[0] += len(board.pieces(chess.BISHOP, chess.WHITE))
    num_pieces[0] += len(board.pieces(chess.KING,   chess.WHITE))
    num_pieces[0] += len(board.pieces(chess.QUEEN,  chess.WHITE))
    num_pieces[0] += len(board.pieces(chess.KNIGHT, chess.WHITE))
    num_pieces[0] += len(board.pieces(chess.ROOK,   chess.WHITE))

    num_pieces[1] += len(board.pieces(chess.PAWN,   chess.BLACK))
    num_pieces[1] += len(board.pieces(chess.BISHOP, chess.BLACK))
    num_pieces[1] += len(board.pieces(chess.KING,   chess.BLACK))
    num_pieces[1] += len(board.pieces(chess.QUEEN,  chess.BLACK))
    num_pieces[1] += len(board.pieces(chess.KNIGHT, chess.BLACK))
    num_pieces[1] += len(board.pieces(chess.ROOK,   chess.BLACK))
                             
    return num_pieces

time: 2.91 ms


In [64]:
# import timeit
# import time

# def _template_func(setup, func):
#     """Create a timer function. Used if the "statement" is a callable."""
#     def inner(_it, _timer, _func=func):
#         setup()
#         _t0 = _timer()
#         for _i in _it:
#             retval = _func()
#         _t1 = _timer()
#         return _t1 - _t0, retval
#     return inner

# timeit._template_func = _template_func

# def foo():
#     time.sleep(1)
#     return 42

# t = timeit.Timer(foo)
# t.timeit(number=1)

time: 367 µs


In [71]:
def run(player1, player2, iterations):
#     df_scoreboard = pd.DataFrame(data={}, columns=['game_result','winner','moves_played', 'remaining_w_pieces','remaining_b_pieces'])    
    scores_list = list()
                                 
    for i in range(iterations):
        
        terminal_state = play_game(player1, player2, visual="svg", pause=0)
#         time = timeit.timeit(play_game(player1, player2, visual="svg", pause=0), number=100)/100
        
        game_hase_winner = terminal_state[0]
        msg = terminal_state[1]
        moves_played = len(terminal_state[2].move_stack)
        remaining_w_pieces = count_pieces(terminal_state[2])[0]
        remaining_b_pieces = count_pieces(terminal_state[2])[1]

#         result_list = (game_hase_winner, msg, moves_played, count_pieces(result[2])[0], count_pieces(result[2])[1], result[3])
        result_list = (game_hase_winner, msg, moves_played, remaining_w_pieces, remaining_b_pieces)
        scores_list.append(result_list)
        
    return scores_list

time: 1.44 ms


### naive evaluation

In [72]:
scores = run(random_player, random_player, 5)

draw: insufficient material
time: 8.85 s


In [73]:
df_naive_eval_scoreboard = pd.DataFrame(data=scores, columns=['game_has_winner','winner','moves_played', 'remain_w_pieces','remaining_b_pieces'])
df_naive_eval_scoreboard

,game_has_winner,winner,moves_played,remain_w_pieces,remaining_b_pieces
0,None,draw: claim,366,3,1
1,True,checkmate: White wins!,321,4,1
2,None,draw: insufficient material,350,2,1
3,None,draw: claim,185,2,5
4,None,draw: insufficient material,364,2,1


time: 6.94 ms


,game_result,winner,moves_played,remain_w_pieces,remaining_b_pieces
0,None,draw: insufficient material,429,3,1


time: 6.85 ms


In [ ]:
def naive_eval(board, move, my_color):
    score = 0
    ## Check some things about this move:
    # score += 10 if board.is_capture(move) else 0
    # To actually make the move:
    board.push(move)
    # Now check some other things:
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 4), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 10), 
                           (chess.KNIGHT, 5),
                           (chess.ROOK, 3)]:
        score += len(board.pieces(piece, my_color)) * value
        score -= len(board.pieces(piece, not my_color)) * value
        # can also check things about the pieces position here
    return score

In [ ]:
def naive_agent(board):
    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()
        # go through board and return a score
        move.score = naive_eval(newboard, move, board.turn)
    moves.sort(key=lambda move: move.score, reverse=True) # sort on score
    return moves[0].uci()

In [ ]:
result = play_game(random_player, naive_agent, visual="svg", pause=0)

In [ ]:
print(result)

In [ ]:
count_pieces(result[2])

### naive evaluation with random heuristic

In [ ]:
def naive_random_heuristic_eval(board, move, my_color):
    score = random.random()
    ## Check some things about this move:
    # score += 10 if board.is_capture(move) else 0
    # To actually make the move:
    board.push(move)
    # Now check some other things:
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 4), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 10), 
                           (chess.KNIGHT, 5),
                           (chess.ROOK, 3)]:
        score += len(board.pieces(piece, my_color)) * value
        score -= len(board.pieces(piece, not my_color)) * value
        # can also check things about the pieces position here
    # Check global things about the board
    score += 100 if board.is_checkmate() else 0
    return score

In [ ]:
def naive_random_heuristic_agent(board):
    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()
        # go through board and return a score
        move.score = naive_random_heuristic_eval(newboard, move, board.turn)
    moves.sort(key=lambda move: move.score, reverse=True) # sort on score
    return moves[0].uci()

In [ ]:
%%time
play_game(random_player, naive_random_heuristic_agent)

### minimax evaluation

In [ ]:
def alphabeta( alpha, beta, depthleft ):
    
    if( depthleft == 0 ):
        return quiesce( alpha, beta )
    for move in board.legal_moves:
        board.push(move)   
        score = -alphabeta( -beta, -alpha, depthleft - 1 )
        board.pop()
        if( score >= beta ):
            return score
        if( score > bestscore ):
            bestscore = score
        if( score > alpha ):
            alpha = score   
    return bestscore



In [ ]:
def minimax_eval(depth, board, move, my_color):
    score = -9999
   
    board.push(move)
   
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 4), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 10), 
                           (chess.KNIGHT, 5),
                           (chess.ROOK, 3)]:
        score += len(board.pieces(piece, my_color)) * value
        score -= len(board.pieces(piece, not my_color)) * value
        # can also check things about the pieces position here
    # Check global things about the board
    
    score += 100 if board.is_checkmate() else 0
    return score

In [ ]:
def minimax_agent(board, depth):
    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()

        move.score = minimax_eval(depth, newboard, move, board.turn)
    moves.sort(key=lambda move: move.score, reverse=True) # sort on score
    return moves[0].uci()

def minimax(depth, board, is_maximizing):
    if(depth == 0):
        return -evaluation(board)
    
    possibleMoves = board.legal_moves
    if(is_maximizing):
        bestMove = -9999
        for x in possibleMoves:
            move = chess.Move.from_uci(str(x))
            board.push(move)
            bestMove = max(bestMove,minimax(depth - 1, board, not is_maximizing))
            board.pop()
        return bestMove
    else:
        bestMove = 9999
        for x in possibleMoves:
            move = chess.Move.from_uci(str(x))
            board.push(move)
            bestMove = min(bestMove, minimax(depth - 1, board, not is_maximizing))
            board.pop()
        return bestMove